In [1]:
from langchain_community.document_loaders import PyPDFLoader 
loader=PyPDFLoader('MSH_Q3_2025_VN.pdf')
docs=loader.load()

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    add_start_index=True
)
chunks=splitter.split_documents(docs)

In [3]:
chunks[1].page_content[200:500]

'nghị GIỮ, giá mục tiêu 42,600 đồng \nCTCP May Sông Hồng (HOSE: MSH) là m ột trong những doanh nghiệp sản xuất hàng may mặc xuất khẩu và   \nchăn ga gối nệm hàng đầu Việt Nam.  \n9T2025 của MSH ghi nhận doanh thu thuần đạt 4,149 tỷ VND (+8% YoY), lợi nhuận sau thuế đạt 468 tỷ VND     \n(+74% YoY), hoàn t'

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os 
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')
emb=OpenAIEmbeddings()
vectorStore=FAISS.from_documents(chunks,emb)


In [ ]:
retriever=vectorStore.as_retriever(
    search_type='mmr',
    search_kwargs={"k":5,"fetch_k":40}
)

In [24]:
# 1) LLM CHAT, không phải OpenAI(text)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_template(
    """Bạn chỉ trả lời DỰA TRÊN Context. Nếu thiếu, nói 'Không tìm thấy thông tin trong tài liệu đã cung cấp.'
# Context:
{context}

# Câu hỏi:
{question}"""
)

def format_docs(docs):
    formatted = []
    for d in docs:
        page = d.metadata.get("page", "?")
        formatted.append(f"[Trang {page}] {d.page_content}")
    return "\n\n".join(formatted)

# 2) Chuỗi runnable: input là STRING question (đỡ phải chuyền dict)
chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),   # nhận chính input string làm question
    }
    | prompt
    | llm
    | StrOutputParser()  # lấy .content ra string
)

print(chain.invoke("Những sự kiện quan trọng của MSH"))


Những sự kiện quan trọng của MSH bao gồm:

1. Đại dịch Covid-19.
2. Hoạt động xuất khẩu phục hồi nhờ sự tăng trưởng trở lại của các thị trường trọng điểm.
3. Tỷ giá tăng làm gia tăng chi phí đầu vào và nhà máy SH10 đi vào hoạt động góp phần tăng doanh thu nhưng không đáng kể khiến LNST giảm 22%.
4. Đơn hàng dồi dào trở lại.
5. Nhà máy Xuân Trường II khởi công.
6. Kinh tế thế giới hồi phục tích cực.
7. Khách hàng lớn New York Company mất khả năng thanh toán.
8. Trump công bố áp thuế đối ứng 46% với VN.
9. Trump hoãn thuế 90 ngày.
10. Trump công bố mức thuế chung 20% sau đàm phán.
